In [67]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
# 填入正确路径
file_path = '/content/drive/MyDrive/Kyoto/Data/kyoto.csv'
df_bloom = pd.read_csv(file_path)
df_bloom.rename(columns={'bloom_date': 'DATE'}, inplace=True)
df_bloom["DATE"] = pd.to_datetime(df_bloom["DATE"], errors="coerce")

<ipython-input-68-bbad8c5b32e1>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_bloom["DATE"] = pd.to_datetime(df_bloom["DATE"], errors="coerce")


In [69]:
df_bloom

location        lat        long  alt  year       DATE  bloom_doy
0      kyoto  35.011983  135.676114   44   812        NaT         92
1      kyoto  35.011983  135.676114   44   815        NaT        105
2      kyoto  35.011983  135.676114   44   831        NaT         96
3      kyoto  35.011983  135.676114   44   851        NaT        108
4      kyoto  35.011983  135.676114   44   853        NaT        104
..       ...        ...         ...  ...   ...        ...        ...
831    kyoto  35.011983  135.676114   44  2020 2020-04-01         92
832    kyoto  35.011983  135.676114   44  2021 2021-03-26         85
833    kyoto  35.011983  135.676114   44  2022 2022-04-01         91
834    kyoto  35.011983  135.676114   44  2023 2023-03-25         84
835    kyoto  35.011983  135.676114   44  2024 2024-04-04         95

[836 rows x 7 columns]

In [70]:
file_path = '/content/drive/MyDrive/Kyoto/Data/Kyoto_weather.csv'
df_weather = pd.read_csv(file_path)

In [71]:
# 计算均温
def calculate_tavg(x, y, eta):
  return eta * x + (1 - eta) * y

df_weather["TAVG"] = df_weather.apply(lambda x: calculate_tavg(x['TMAX'], x['TMIN'], 0.5), axis=1)

# 提取年份
df_weather["DATE"] = pd.to_datetime(df_weather["DATE"], errors="coerce")
df_weather["year"] = df_weather["DATE"].dt.year

# 提取 DOY（一年中的第几天）
df_weather["DOY"] = df_weather["DATE"].dt.dayofyear

# 合并 bloom_doy 数据
df_weather = df_weather.merge(df_bloom[["year", "bloom_doy"]], on="year", how="left")

df_weather_2025 = df_weather[df_weather["year"] == 2025]
df_weather = df_weather[df_weather["year"] != 2025]

In [72]:
df_weather_avaliable = df_weather[df_weather["DOY"] <= df_weather["bloom_doy"]]

# 设定基础温度（通常为 5°C）
BASE_TEMP = 5

# 计算每日 GDD：如果 TAVG < BASE_TEMP，则为 0
# df_weather_avaliable["GDD"] = np.maximum(df_weather_avaliable["TAVG"] - BASE_TEMP, 0)
df_weather_avaliable["GDD"] = df_weather_avaliable["TAVG"] - BASE_TEMP

# 确保DOY升序排列
df_weather_avaliable = df_weather_avaliable.sort_values(by=["year", "DOY"]).reset_index(drop=True)

# 计算按年份累加的GDD
df_weather_avaliable["GDD_cumsum"] = df_weather_avaliable.groupby("year")["GDD"].cumsum()

df_weather_avaliable["PRCP_cumsum"] = df_weather_avaliable.groupby("year")["PRCP"].cumsum()

df_weather_avaliable

<ipython-input-72-a0af7d45b101>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather_avaliable["GDD"] = df_weather_avaliable["TAVG"] - BASE_TEMP


DATE       TMAX  PRCP       TMIN       TAVG  year  DOY  bloom_doy  \
0   2020-01-01   9.700000     5   3.248772   6.474386  2020    1       92.0   
1   2020-01-02  12.300000     0   1.200000   6.750000  2020    2       92.0   
2   2020-01-03  12.300000     0   2.900000   7.600000  2020    3       92.0   
3   2020-01-04  11.800000     0   3.153369   7.476685  2020    4       92.0   
4   2020-01-05   7.300000    30   3.288032   5.294016  2020    5       92.0   
..         ...        ...   ...        ...        ...   ...  ...        ...   
427 2024-03-27  15.800000     0   7.784992  11.792496  2024   87       95.0   
428 2024-04-01  17.113030     0   8.006337  12.559684  2024   92       95.0   
429 2024-04-02  17.596364     0   7.400000  12.498182  2024   93       95.0   
430 2024-04-03  18.260000   160  13.100000  15.680000  2024   94       95.0   
431 2024-04-04  18.100000   109   9.738603  13.919302  2024   95       95.0   

           GDD  GDD_cumsum  PRCP_cumsum  
0     1.474386    1.474386            5  
1     1.750000    3.224386            5  
2     2.600000    5.824386            5  
3     2.476685    8.301070            5  
4     0.294016    8.595087           35  
..         ...         ...          ...  
427   6.792496  181.301065         2722  
428   7.559684  188.860749         2722  
429   7.498182  196.358930         2722  
430  10.680000  207.038930         2882  
431   8.919302  215.958232         2991  

[432 rows x 11 columns]

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 选择用于机器学习的特征
FEATURES = ["GDD_cumsum", "TMAX", "TMIN", "TAVG", "PRCP_cumsum"] #
TARGET = "bloom_doy"

# 只保留开花当天的数据（bloom_doy 对应的 DOY）
df_final = df_weather_avaliable[
    (df_weather_avaliable["DOY"] >= df_weather_avaliable["bloom_doy"] - 5) &
    (df_weather_avaliable["DOY"] <= df_weather_avaliable["bloom_doy"] + 5)
]


# 准备数据
X = df_final[FEATURES]
y = df_final[TARGET]

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集 & 测试集（80% 训练, 20% 测试）
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [74]:
print("Filtered df_final shape:", df_final.shape)


Filtered df_final shape: (28, 11)


In [75]:
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_test shape: (6, 5)
y_test shape: (6,)


In [76]:
# 导入所需的包
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


# 初始化模型
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(max_depth=3),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=2, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=20, random_state=42),
    "Ridge Regression": Ridge(alpha=1.0),
    "Bayesian Ridge": BayesianRidge(),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=3),
    "SVR": SVR(kernel="rbf", C=1.0, epsilon=0.1),
    "Lasso Regression": Lasso(alpha=0.01)
}

# 训练和评估模型
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "R²": r2}

# 显示结果
df_results = pd.DataFrame(results).T
print(df_results)

                         MSE        R²
Linear Regression  15.483394  0.118035
Decision Tree       0.083333  0.995253
Random Forest       6.480259  0.630871
XGBoost             0.025602  0.998542
Ridge Regression   15.353389  0.125440
Bayesian Ridge     16.109381  0.082377
KNN Regressor      12.888889  0.265823
SVR                18.257644 -0.039992
Lasso Regression   15.449311  0.119976


In [77]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# 读取数据
df_weather_available = df_weather.copy()  # 假设 df_weather 是你的原始天气数据

# 计算累积 GDD（生长度日）
BASE_TEMP = 5
df_weather_available["GDD"] = np.maximum(df_weather_available["TAVG"] - BASE_TEMP, 0)
df_weather_available["GDD_cumsum"] = df_weather_available.groupby("year")["GDD"].cumsum()

# 计算累积降水量
df_weather_available["PRCP_cumsum"] = df_weather_available.groupby("year")["PRCP"].cumsum()

# 新增特征
df_weather_available["TMAX_TMIN_DIFF"] = df_weather_available["TMAX"] - df_weather_available["TMIN"]  # 温差
df_weather_available["AVG_GDD_CUMSUM"] = df_weather_available["GDD_cumsum"] / df_weather_available["DOY"]  # 累积GDD的平均增长速率
df_weather_available["PRCP_LAG1"] = df_weather_available["PRCP_cumsum"].shift(1).fillna(0)  # 前一天的降水量
df_weather_available["GDD_DIFF"] = df_weather_available["GDD_cumsum"].diff().fillna(0)  # GDD日变化

# 只保留开花前后三天数据
df_final = df_weather_available[
    (df_weather_available["DOY"] >= df_weather_available["bloom_doy"] - 3) &
    (df_weather_available["DOY"] <= df_weather_available["bloom_doy"] + 3)
]

# 选择特征和目标变量
FEATURES = ["GDD_cumsum", "TMAX", "TMIN", "TAVG", "PRCP_cumsum",
            "TMAX_TMIN_DIFF", "AVG_GDD_CUMSUM", "PRCP_LAG1", "GDD_DIFF"]
TARGET = "bloom_doy"

X = df_final[FEATURES]
y = df_final[TARGET]

# 标准化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 定义模型
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(max_depth=3),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=2, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=20, random_state=42),
    "Ridge Regression": Ridge(alpha=1.0),
    "Bayesian Ridge": BayesianRidge(),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=3),
    "SVR": SVR(kernel="rbf", C=1.0, epsilon=0.1),
    "Lasso Regression": Lasso(alpha=0.01)
}

# 训练和评估模型
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "R²": r2}

# 显示结果
df_results = pd.DataFrame(results).T
print(df_results)


                         MSE        R²
Linear Regression   6.960578  0.356475
Decision Tree       6.234947  0.423562
Random Forest       2.230549  0.793779
XGBoost             5.142962  0.524519
Ridge Regression    5.022195  0.535684
Bayesian Ridge      5.070638  0.531205
KNN Regressor      13.761905 -0.272327
SVR                16.958640 -0.567874
Lasso Regression    5.554537  0.486467


In [78]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
# from tensorflow.keras.optimizers import Adam
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import train_test_split

# # 📌 增加特征
# df_weather_available["TAVG_rolling"] = df_weather_available.groupby("year")["TAVG"].transform(lambda x: x.rolling(7, min_periods=1).mean())
# df_weather_available["TAVG_anomaly"] = df_weather_available["TAVG"] - df_weather_available.groupby("year")["TAVG"].transform("mean")
# df_weather_available["GDD_anomaly"] = df_weather_available["GDD_cumsum"] - df_weather_available.groupby("year")["GDD_cumsum"].transform("mean")

# # 📌 选择特征
# FEATURES = ["GDD_cumsum", "TMAX", "TMIN", "TAVG", "PRCP_cumsum", "TAVG_rolling", "TAVG_anomaly", "GDD_anomaly"]
# TARGET = "bloom_doy"

# X = df_weather_available[FEATURES].values
# y = df_weather_available[TARGET].values

# # 📌 数据标准化
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # 📌 LSTM 需要 3D 输入
# sequence_length = 15
# X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1]))

# # 📌 训练集 & 测试集划分
# X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# # 📌 生成时间序列数据
# X_train_seq = np.array([X_train[i - sequence_length:i] for i in range(sequence_length, len(X_train))])
# y_train_seq = np.array(y_train[sequence_length:])

# X_test_seq = np.array([X_test[i - sequence_length:i] for i in range(sequence_length, len(X_test))])
# y_test_seq = np.array(y_test[sequence_length:])

# # 🚨 修正数据形状
# X_train_seq = X_train_seq.reshape(X_train_seq.shape[0], sequence_length, X_train.shape[1])
# X_test_seq = X_test_seq.reshape(X_test_seq.shape[0], sequence_length, X_test.shape[1])

# # 📌 LSTM 模型
# model = Sequential([
#     Bidirectional(LSTM(128, return_sequences=True, input_shape=(sequence_length, X_train.shape[1]))),
#     Dropout(0.2),
#     Bidirectional(LSTM(64, return_sequences=True)),
#     Dropout(0.2),
#     LSTM(32, return_sequences=False),
#     Dense(16),
#     Dense(1)
# ])

# # 📌 编译模型
# optimizer = Adam(learning_rate=0.0005)
# model.compile(optimizer=optimizer, loss="mse")

# # 📌 训练模型
# history = model.fit(X_train_seq, y_train_seq, validation_data=(X_test_seq, y_test_seq), epochs=100, verbose=1)

# # 📌 预测
# y_pred = model.predict(X_test_seq)

# # 📌 计算 MSE 和 R²
# mse = mean_squared_error(y_test_seq, y_pred)
# r2 = r2_score(y_test_seq, y_pred)

# print(f"LSTM Model Performance:")
# print(f"Mean Squared Error (MSE): {mse:.4f}")
# print(f"R² Score: {r2:.4f}")


In [84]:
from statsmodels.tsa.arima.model import ARIMA

# 只用 2020-2024 的 TAVG（平均气温） 训练 ARIMA 模型
df_train = df_weather[df_weather["year"] < 2025][["DATE", "TAVG"]]
df_train.set_index("DATE", inplace=True)

# 训练 ARIMA 模型
model = ARIMA(df_train, order=(5,1,0))  # ARIMA(5,1,0) 代表自回归+差分
model_fit = model.fit()

# 预测 2025 年 2-4 月的温度
future_dates = pd.date_range("2025-02-20", "2025-04-30")
forecast = model_fit.forecast(steps=len(future_dates))

# 创建预测数据框
df_forecast = pd.DataFrame({"DATE": future_dates, "TAVG": forecast})
df_forecast["year"] = 2025
df_forecast["DOY"] = df_forecast["DATE"].dt.dayofyear

# 把预测数据合并到 df_weather
df_weather = pd.concat([df_weather, df_forecast], ignore_index=True)


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsm

In [90]:
year = 2025  # 选择预测的年份（可以改成 2025）

# **选择最新一年的气象数据**
df_test = df_weather[df_weather["year"] == year].copy()

# **设定基础温度（通常为 5°C）**
BASE_TEMP = 5

# **计算每日 GDD**
df_test["GDD"] = np.maximum(df_test["TAVG"] - BASE_TEMP, 0)
df_test["GDD_cumsum"] = df_test.groupby("year")["GDD"].cumsum()
df_test["PRCP_cumsum"] = df_test.groupby("year")["PRCP"].cumsum()
df_test["TMAX_TMIN_DIFF"] = df_test["TMAX"] - df_test["TMIN"]
df_test["AVG_GDD_CUMSUM"] = df_test["GDD_cumsum"].rolling(window=7, min_periods=1).mean()
df_test["PRCP_LAG1"] = df_test["PRCP"].shift(1).fillna(0)
df_test["GDD_DIFF"] = df_test["GDD"].diff().fillna(0)


latest_data = df_test.iloc[-1][FEATURES].values.reshape(1, -1)

# **标准化**
latest_data_scaled = scaler.transform(latest_data)

# **选择最优模型**
best_model = models["XGBoost"]  # 换成 R² 最高的模型

# **进行预测**
predicted_bloom_doy = best_model.predict(latest_data_scaled)[0]

print(f"🔮 预测 {year} 年的樱花开花日期 (DOY): {int(predicted_bloom_doy)}")


🔮 预测 2025 年的樱花开花日期 (DOY): 83


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Random Forest: 86 \\
Decision Tree: 84 \\
XGBoost: 83